<a href="https://colab.research.google.com/github/ivs-math/BinPacking/blob/main/Clasificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Categorized Bin Packing 


---


This notebook discuses the solution to a particular bin packing problem that requires previous item classification. 

## Objective:
 **Determine the number of cold bags and baskets for each order.**

## Context
Today, orders are not being packed in a very efficient way, thus, the warehouse chief asked you to create an algorithm that can solve for any set of products how many baskets and cold bags are needed to correctly pack an order. 

## Conditions
Each order contains different products that have diverse characteristics.

Each of the products has:

**Dimensions** *(height, length, width)* in centimeters and *Weight* in grams

A **category** that determines if the product can be mixed with other products:

* *Food*

* *Toilet*

* *Pets*

A **storage type** that determines the type of package that will be used for the product

* *Dry*

* *Refrigerated*

* *Frozen*

There are some constraints associated with the type of package to be used when packing a product:

* All Dry products must be packed in a baskets

* All Refrigerated and Frozen products must be packed in a cold bag

* Products cannot be mixed if their categories are different, for example, a food product cannot be packed in the same basket that was used to pack a pet product, and also, a toilet product cannot be packed in the same basket that was used to pack a pet product, and so on.

The baskets have the following characteristics:

* Length: 50 centimeters

* Width: 40 centimeters

* Height: 60 centimeters

* Total weight that it can resit: 25 kilograms


The cold bags have the following characteristics:

* Length: 23 centimeters

* Width: 37 centimeters

* Height: 18 centimeters

* Total weight that it can resit: 5 kilograms

## Dataset

The given dataset is separated in different csv files:

* **orders**: it contains the orders that need the calculation of baskets and cold bags. Also, you can see when the order must be delivered and which warehouse received the order.

* **order_products**: contains what products and how many units of it were on an order from the file “orders”.

* **store_products**: contains information about the storage of the products and a marketing category.

* **products**: contains information about the products, its dimensions and weight, as well as their names and the can_mix category.




#**Solution**

---
\\
##**Bin Packing**

In order to solve the bin packing problem, we will use a *PypI* library named *binpacking*. This package contains greedy algorithms to solve two typical bin packing problems,  sorting items into a constant number of bins, and sorting items into a low number of bins of constant size.



> https://pypi.org/project/binpacking



This bin approximation will be used to distribute the volume from articles of a single category and storage type. Then, if any of the bin exceeds the maximum weight, the algorithm will take the heaviest item to a new bin.

This approximation prioritizes the volume over the weight, considering that it is more likely to exceed the maximum volume when optimizing the weight distribution.


In [36]:
!pip install binpacking

  Created wheel for binpacking: filename=binpacking-1.5.2-py3-none-any.whl size=10106 sha256=8a634c98071e0e1abd8be38a61388d3d43dd36496d591842c17ce1e638b97cde
  Stored in directory: /root/.cache/pip/wheels/32/bd/69/f86ce791ee53077576de9bee2bc8944ce187f445b416318488
Successfully built binpacking


##**Classification**

Before any bin is created ,we need to take all the items in an order and classify them in one of the following groups:

* Dry-Pets (DP)
* Dry-Toilet (DT)
* Dry-Food (DF)
* Refrigerated-Pets (RP)
* Refrigerated-Toilet (RT)
* Refrigerated-Food (RF)

The bin packing algorithm will be applied only after this classification is complete.

From now we will solve the problem for the order **1781560**. In the final part we will generalize the solution for any given order.

> We define a pandas dataframe for each csv. This will allow us to access the data efficiently.

In [5]:
import pandas as pd
df_products=pd.read_csv('products.csv')
df_order_prod=pd.read_csv('order_products.csv')
df_orders=pd.read_csv('orders.csv')
df_store_prod=pd.read_csv('store_products.csv')

>  Let's take all the products in order_products with the same order_id.

In [6]:
df_list=df_order_prod[df_order_prod['order_id']==1781562]
df_list=df_list.reset_index(drop=True)

> Then we will search for the storage and refrigeration type.

In [7]:
#Classify the Storage

Storage=[]
for x in df_list['store_product_id']:
  z=df_store_prod.loc[df_store_prod['store_product_id'] == x]
  l=z['storage'].values.tolist()
  Storage.append(l[0])

df_list.insert(1,"Storage",Storage)


In [8]:
#Classify the Category
Prod_id=[]

for x in df_list['store_product_id']:
  z=df_store_prod.loc[df_store_prod['store_product_id'] == x]
  l=z['product_id'].values.tolist()
  Prod_id.append(l[0])

df_list.insert(1,"product_id",Prod_id)

Mix=[]
for x in df_list['product_id']:
  z=df_products.loc[df_products['product_id'] == x]
  l=z['can_mix'].values.tolist()
  Mix.append(l[0])

df_list.insert(1,"can_mix",Mix)


>Now that each item is classified, we will calculate its weight and volume.

In [9]:
volume=[]
weight=[]
for x in df_list['product_id']:
  z=df_products.loc[df_products['product_id'] == x]
  len=z['length'].values.tolist()
  wid=z['width'].values.tolist()
  hei=z['height'].values.tolist()
  wei=z['weight'].values.tolist()
  vol=len[0]*wid[0]*hei[0]
  volume.append(vol)
  weight.append(wei[0])

df_list.insert(1,"volume",volume)
df_list.insert(1,"weight",weight)


df_list


,order_id,weight,volume,can_mix,product_id,Storage,store_product_id,quantity
0,1781562,2500.000,36046.08,Food,181808.0,Seco,16118,1
1,1781562,508.000,2730.00,Food,1320.0,Seco,647,2
2,1781562,1628.000,2673.00,Food,1877.0,Seco,742,1
3,1781562,492.000,700.00,Food,173582.0,Refrigerado,1170,1
4,1781562,418.000,1664.00,Food,174323.0,Seco,4542,1
5,1781562,1022.000,1560.00,Food,172897.0,Refrigerado,879,1
6,1781562,1022.000,1560.00,Food,172898.0,Refrigerado,880,1
7,1781562,1022.000,1560.00,Food,172955.0,Refrigerado,884,2
8,1781562,6820.000,8280.00,Food,178184.0,Seco,10414,1
9,1781562,0.373,550.00,Toilet,178230.0,Seco,10490,1


>In this step I will create dictionaries to strore each product's volume classified in one of the six categories. 
If the quantity of a product in the order is 2 or more, the volume will be stored that many times. The key for each product will be its "strore_product_id".

In [94]:
st=df_list['Storage'].values.tolist()
mx=df_list['can_mix'].values.tolist()
key=df_list['store_product_id'].values.tolist()
qnt=df_list['quantity'].values.tolist()
vol=df_list['volume'].values.tolist()
DF={}
DT={}
DP={}
RF={}
RT={}
RP={}

for x, y,k,q,v in zip(st, mx, key, qnt, vol):
  lst=[]
  if (x,y)==('Seco','Food'):
    for x in range(0,q):
      lst.append(v)
    DF[k]=lst
    DF['clase']='DF'
  if (x,y)==('Seco','Toilet'):
    for x in range(0,q):
      lst.append(v)
    DT[k]=lst
    DT['clase']='DT'
  if (x,y)==('Seco','Pets'):
    for x in range(0,q):
      lst.append(v)
    DP[k]=lst
    DP['clase']='DP'
  if ((x,y)==('Congelado','Food')) or ((x,y)==('Refrigerado','Food')):
    for x in range(0,q):
      lst.append(v)
    RF[k]=lst
    RF['clase']='RF'   
  if ((x,y)==('Congelado','Toilet')) or ((x,y)==('Refrigerado','Toilet')):
    for x in range(0,q):
      lst.append(v)
    RT[k]=lst
    RT['clase']='RT'
  if ((x,y)==('Congelado','Pets')) or ((x,y)==('Refrigerado','Pets')):
    for x in range(0,q):
      lst.append(v)
    RP[k]=lst
    RP['clase']='DT'  

    

print('DF:',DF)
print('DT:',DT)
print('DP:',DP)
print('RF:',RF)
print('RT:',RT)
print('RP:',RP)



DF: {16118: [36046.08], 'clase': 'DF', 647: [2730.0, 2730.0], 742: [2673.0], 4542: [1664.0], 10414: [8280.0], 14931: [9610.0], 1249: [8316.0], 30671: [2197.0], 9306: [2352.0], 1102: [2000.0]}
DT: {10490: [550.0], 'clase': 'DT', 6443: [4500.0], 13422: [3640.0]}
DP: {}
RF: {1170: [700.0], 'clase': 'RF', 879: [1560.0], 880: [1560.0], 884: [1560.0, 1560.0]}
RT: {}
RP: {}


>Since we have the dictionaries with the volumes of each product. The following step is to generate the bin packing for each classification. Remember that this bin packing is only based in volume.

In [96]:
#Let's bin
import binpacking
order=[DF,DT,DP,RF,RP,RT]
A=[]
for d in order:
  for values in d.values():
    if type(values)==list:
      A=A+values
  
  


  if A and (d==DT or d==DF or d==DP):
    print('Clase:', d['clase'])
    print('Objects',A)
    bins=binpacking.to_constant_volume(A,12000)
    print('Bins:',bins)
    print('#Baskets--->',bins.__len__())
  if A and (d==RT or d==RP or d==RF):
    print('Clase',d['clase'])
    print('Objects',A)
    bins=binpacking.to_constant_volume(A,15318)
    print('Bins:',bins)
    print('#Cold Bags--->',bins.__len__())   
  
  A=[]

Clase: DF
Objects [36046.08, 2730.0, 2730.0, 2673.0, 1664.0, 8280.0, 9610.0, 8316.0, 2197.0, 2352.0, 2000.0]
Bins: [[36046.08], [9610.0, 2352.0], [8316.0, 2730.0], [8280.0, 2730.0], [2673.0, 2197.0, 2000.0, 1664.0]]
#Baskets---> 5
Clase: DT
Objects [550.0, 4500.0, 3640.0]
Bins: [[4500.0, 3640.0, 550.0]]
#Baskets---> 1
Clase RF
Objects [700.0, 1560.0, 1560.0, 1560.0, 1560.0]
Bins: [[1560.0, 1560.0, 1560.0, 1560.0, 700.0]]
#Cold Bags---> 1
